# **1. Install Dependencies**



In [17]:
!pip install pinecone langchain-community streamlit ollama langchain-ollama langchain_pinecone langchain

In [3]:
from ollama import Client

client = Client()

# **2. Instantiate an instance of the Pinecone client.**

In [28]:
import os
from pinecone import Pinecone



# Get your API key at app.pinecone.io
api_key = os.environ.get("PINECONE_API_KEY") or "pcsk_476Mbq_K7Z3aCy8BqmFGYrsbHHwhoeRLcfz9yWg8qbpmZMFBEnCMZR6PsPqPiuzDs9neEe"

# Instantiate the Pinecone client
pc = Pinecone(api_key=api_key)

# Giving our index a name
index_name = "chatbot-index"



In [30]:
from pinecone import ServerlessSpec, CloudProvider, AwsRegion, Metric

# Create an index if it doesn't exist
if index_name not in pc.list_indexes().names():
   pc.create_index(
        name=index_name,
        dimension=1024,  # Use 1536 for `text-embedding-3-large`
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1")
    )
index = pc.Index(index_name)
print("Pinecone index initialized!")

Pinecone index initialized!


In [25]:
# Delete the index
# pc .delete_index("chatbot-index")

# **3. Loading Data from PDFs and CSVs Or Websites**

* WebBaseLoader to load all text from HTML webpages into a document format that we can use downstream.

In [32]:
#set user agent variable to identify requests
import os

os.environ["USER_AGENT"] = "MyLangChainBot/1.0"

In [34]:
from langchain_community.document_loaders import WebBaseLoader, CSVLoader, PyPDFLoader

# load data from website
def WebsiteLoader(urls):
    loader = WebBaseLoader(urls)
    return loader.load()

urls = ["https://nextjs.org/","https://vuejs.org/"]
website_docs = WebsiteLoader(urls)

In [35]:
# load data from csv files

"""
def load_csvs(file_paths):
    docs = []
    for file_path in file_paths:
        docs.extend(CSVLoader(file_path=file_path).load())
    return docs


csv_docs = load_csvs([""])
"""


'\ndef load_csvs(file_paths):\n    docs = []\n    for file_path in file_paths:\n        docs.extend(CSVLoader(file_path=file_path).load())\n    return docs\n\n\ncsv_docs = load_csvs([""])\n'

In [ ]:
# load data from pdf files

"""
def load_pdfs(pdf_files):
    docs = []
    for pdf_file in pdf_files:
        docs.extend(PyPDFLoader(file_path=pdf_file).load())

pdf_docs = load_pdfs([""])

"""

In [38]:
website_docs

[Document(metadata={'source': 'https://nextjs.org/', 'title': 'Next.js by Vercel - The React Framework', 'description': 'Next.js by Vercel is the full-stack React framework for the web.', 'language': 'en'}, page_content="Next.js by Vercel - The React FrameworkSkip to contentSearch documentation...Search...⌘KShowcaseDocsBlogTemplatesEnterpriseSearch documentation...Search...⌘KDeployLearnThe React Framework for the WebUsed by some of the world's largest companies, Next.js enables you to create high-quality web applications with the power of React components.Get StartedLearn Next.js▲ ~ npx create-next-app@latestWhat's in Next.js?Everything you need to build great products on the web.Data FetchingMake your React component async and await your data. Next.js supports both server and client data fetching.Server ActionsRun server code by calling a function. Skip the API. Then, easily revalidate cached data and update your UI in one network roundtrip.Advanced Routing & Nested LayoutsCreate rout

4. Chunking: breaking the data into small chunks

  * RecursiveCharacterTextSplitter is a text-splitting tool that breaks documents into smaller parts while maintaining logical relationships.
  * chunk_size=500 → Each chunk will have a maximum of 500 characters.
  * chunk_overlap=100 → Chunks will have 100-character overlap to maintain context.
  * length_function=len → Uses character length to determine chunk size.
  * is_separator_regex=False → The separator used for splitting is not a regex pattern.

In [40]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,chunk_overlap=100, length_function=len,
    is_separator_regex=False,
)

# all_documents = website_docs + csv_docs + pdf_docs -> merges text data from websites, CSV files, and PDFs into a single list for processing.
all_documents = website_docs

splited_documents = text_splitter.split_documents(all_documents)

In [42]:
len(splited_documents)

24

# **5. Embedding Documents and Storing in Pinecone**



*  import OllamaEmbeddings, a model for converting text into vector representations (numerical embeddings).
* Uses Ollama’s mxbai-embed-large model to generate vector embeddings for text.
These embeddings represent textual content in a high-dimensional space, enabling similarity search.
* PineconeVectorStore is a vector database that efficiently stores and retrieves vectorized documents.
* embedding=embeddings → Uses Ollama embeddings to convert text into vectors before storage.
* index=index → Specifies the Pinecone index where vectors will be stored.
* Converts splited_documents into vectors using OllamaEmbeddings.
* Stores the vectorized documents in Pinecone for fast semantic search and retrieval.




In [44]:
from langchain_ollama.embeddings import OllamaEmbeddings
from langchain_pinecone import PineconeVectorStore

embeddings  = OllamaEmbeddings(
  model='mxbai-embed-large'
)

vectorstore = PineconeVectorStore(embedding=embeddings, index=index )

vectorstore.add_documents(documents=splited_documents)

['493634df-b809-4986-acd7-f12df66de9f0',
 'a1137871-b1fa-40a9-a169-b4acaca77f41',
 '57a1186b-cafe-48da-8723-10e80fb2d639',
 '70c91262-d4a8-492a-b2b0-12cfa9426b4e',
 'ab0851b4-6f63-49c1-8324-8d65be649d84',
 'd7c4f99b-ffd4-4070-aee7-d9be71b51150',
 'f65eeb80-4220-4671-a8d7-16b962491ee3',
 '0a3f6d5c-99a1-4be2-8e0f-a526479643c7',
 '583008f9-c23d-4424-b824-50fa9f54728e',
 '04bc21de-a24f-44d7-baa1-cd5612da0267',
 'd1b680fb-f529-49ed-9330-ad4833884969',
 'f5d2e82e-b9a1-4431-bb65-fe4c2a3ad902',
 '59995c3d-9155-4964-8c1c-cbf333395c93',
 '71be8f37-349a-49e8-b8b5-4a0fa8ded246',
 '6b34b851-20dd-4e54-8d68-049241ba792c',
 '5083bb81-2254-4330-be3d-4364e12b2b28',
 '8d9f676d-f4f9-404b-9ea8-ceeb81edaa6e',
 '9783bf6b-09ef-4268-8af8-20445f136eb7',
 '6336be50-770e-4caa-8064-10d3ebec0df5',
 '3189b67f-b49d-428d-8bc8-0e22f38c1a40',
 '7fe8c42f-c72d-4430-8628-96284469f918',
 '6ad1d763-a3c5-42c7-8192-6d3cd007d9dd',
 '7d5917b3-102e-4e6d-b5b0-adf4c5d9b789',
 'bfd212a5-224d-48bf-97e8-aaa9adab2f68']

In [64]:
from langchain_ollama import ChatOllama

llm = ChatOllama(model="llama3")

In [70]:
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_ollama import ChatOllama

llm = ChatOllama(model="llama3")

retriever = vectorstore.as_retriever(search_type="similarity")

prompt = PromptTemplate.from_template(
    """Use the following context to answer the question.

Context:
{context}

Question:
{question}
"""
)

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

response = chain.invoke("tell me about Next.js")
print(response)

Next.js is a full-stack React framework for building web applications. According to its documentation, it provides everything you need to build great products on the web.

Some of its key features include:

1. **Data Fetching**: Make your React component async and await your data. Next.js supports both server and client-side data fetching.
2. **Server Actions**: Run server code by calling a function. This allows for easy revalidation of cached data and updating the UI in one network roundtrip.
3. **Advanced Routing & Nested Layouts**: Create routes using the file system, including support for more advanced routing patterns and UI layouts.
4. **CSS Support**: Style your application with ease.

Additionally, Next.js offers:

1. **Server Rendering**: Enable flexible rendering and caching options, including Incremental Static Regeneration (ISR) on a per-page level.
2. **Turbopack**: An incremental bundler optimized for JavaScript and TypeScript, written in Rust, which is used to build the 

In [74]:
import streamlit as st
import os
from pinecone import Pinecone

from langchain_ollama import ChatOllama
from langchain_ollama.embeddings import OllamaEmbeddings
from langchain_pinecone import PineconeVectorStore

from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# ----------------- Pinecone Setup ----------------- #
PINECONE_API_KEY = "pcsk_476Mbq_K7Z3aCy8BqmFGYrsbHHwhoeRLcfz9yWg8qbpmZMFBEnCMZR6PsPqPiuzDs9neEe"
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY

pc = Pinecone(api_key=PINECONE_API_KEY)
index = pc.Index("chatbot-index")  # Make sure this index exists and has the correct dimension (1024)

# ----------------- Streamlit UI ----------------- #
st.set_page_config(layout="wide")
st.title("My Modern Local Chatbot")

st.sidebar.header("Settings")
MODEL = st.sidebar.selectbox(
    "Choose a Model",
    ["llama3.2", "deepseek-r1:1.5b"],
    index=0
)
MAX_HISTORY = st.sidebar.number_input("Max History", 1, 10, 2)

# ----------------- Session State ----------------- #
if "chat_history" not in st.session_state:
    st.session_state.chat_history = []

# ----------------- LangChain Components ----------------- #
llm = ChatOllama(
    model=MODEL,
    streaming=True  # Token streaming enabled
)

embeddings = OllamaEmbeddings(model="mxbai-embed-large")

vectorstore = PineconeVectorStore(
    index=index,
    embedding=embeddings
)

retriever = vectorstore.as_retriever(search_type="similarity")

# ----------------- Prompt Template ----------------- #
prompt_template = PromptTemplate.from_template(
    """You are a helpful assistant.
Use the context below to answer the question.
If the answer is not in the context, say so clearly.

Context:
{context}

Question:
{question}

Answer:"""
)

# ----------------- LCEL Chain ----------------- #
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt_template
    | llm
    | StrOutputParser()
)

# ----------------- Display Chat History ----------------- #
for msg in st.session_state.chat_history:
    with st.chat_message(msg["role"]):
        st.markdown(msg["content"])

# ----------------- Trim History ----------------- #
def trim_history():
    while len(st.session_state.chat_history) > MAX_HISTORY * 2:
        st.session_state.chat_history.pop(0)

# ----------------- Chat Input ----------------- #
if question := st.chat_input("Ask something"):
    st.session_state.chat_history.append({"role": "user", "content": question})

    with st.chat_message("user"):
        st.markdown(question)

    trim_history()

    with st.chat_message("assistant"):
        response_box = st.empty()
        streamed_text = ""

        # -------- Retrieve Documents (NEW retriever API) -------- #
        docs = retriever(question)  # NEW: retriever is callable
        context_text = "\n\n".join([doc.page_content for doc in docs])

        # -------- Prepare LCEL input -------- #
        lcel_input = {"context": context_text, "question": question}

        # -------- Stream Tokens -------- #
        for token in chain.stream(lcel_input):
            streamed_text += token
            response_box.markdown(streamed_text)

        # -------- Show Sources -------- #
        if docs:
            with st.expander("📚 Sources"):
                for i, doc in enumerate(docs, 1):
                    source = doc.metadata.get("source", "Unknown")
                    st.markdown(f"**{i}.** {source}")

        # -------- Save to Session State -------- #
        st.session_state.chat_history.append({"role": "assistant", "content": streamed_text})

        trim_history()


2026-01-29 12:42:24.554 
  command:

    streamlit run C:\Users\PC\anaconda3\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2026-01-29 12:42:24.554 Session state does not function when running a script without `streamlit run`
